In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-11-24 16:45:29.985266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-24 16:45:29.985593: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-24 16:45:29.985807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1739989904168012208
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 124649472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10678667843814264539
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# 딥 러닝 모델 로드
model = load_model('../snack_data/snack_model_copy1_3000-20231125T063742Z-001/snack_model_copy1_3000/')  # 모델 파일의 경로를 지정

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    # 웹캠에서 프레임 읽기
    ret, frame = cap.read()

    cv2.putText(frame, "Snack Box Empty, Full Check", (400,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    # 전처리: 크기 조정 등 모델의 입력에 맞게 프레임을 전처리해야 함
    # 이 부분은 모델이 학습된 입력 형식에 맞게 적절히 수정해주세요.
    processed_frame = cv2.resize(frame, (220, 180))
    processed_frame = processed_frame / 255.0  # 모델이 학습할 때 정규화되었는지 확인

    # 모델 예측
    input_array = np.expand_dims(processed_frame, axis=0)  # 모델은 배치 차원을 기대하므로 차원 확장
    predictions = model.predict(input_array)

    # 예측 결과
    # 이 부분은 예측 결과를 어떻게 처리할지에 따라 수정해야 합니다.
    # 여기서는 간단하게 클래스별로 확률을 출력합니다.
    class_probabilities = predictions[0]
    class_index = np.argmax(class_probabilities)

    # class_labels 리스트에 "OFF"와 "ON"을 순서대로 넣어둔다고 가정합니다.
    class_labels = ["Empty", "Full"]

    # class_index를 기반으로 클래스 레이블 설정
    class_label = class_labels[class_index]

    # 확률을 퍼센트로 변환하여 문자열 구성
    class_probability_percent = class_probabilities[class_index] * 100
    # class_label_text = f"snack Box: {class_label}, Probability(%): {class_probability_percent:.4f}"



    class_label = f"snack Box: {class_index}, Probability(%): {class_probabilities[class_index]:.4f}"


    # 결과를 화면에 출력
    cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 10, 0), 2)

    # 화면에 프레임 표시
    cv2.imshow('Webcam', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 17ms/step
